#                                            Prodution Analysis

#### Importing dependecies & setting up display enviroment 

In [1]:
import pandas as pd 
import openpyxl
import matplotlib.pyplot as plt
import os 
import numpy as np
import calendar
%matplotlib inline
pd.set_option('display.max_columns', 1000)  # or 1000
pd.set_option('display.max_rows', 1000)  # or 1000
pd.set_option('display.max_colwidth', 199)  # or 199

#### importing Data and converting them into data frame objects

In [110]:
path_ = r'C:\Users\peter.vini\Desktop\python\Pandas-analysis\production estimate\\'
def read_DF(path_):
    customer_DF,csmaterial_DF,material_DF,prod_DF\
    = [pd.read_csv(path_ + DF) for DF in os.listdir(path_) if os.path.exists(path_) ]
    return customer_DF,csmaterial_DF,material_DF,prod_DF
customer_DF,csmaterial_DF,material_DF,prod_DF = read_DF(path_)

#### Data Preparation  
##### Removing duplicates and not needed data 
##### merging the data frames (data_DF--> Preapred data frame for analysis)

In [111]:
#--prod_DF--
prod_DF.drop(['TUBETYPE','OLDMATNO','PWOAverage','PWOCount'],axis='columns',inplace=True)
prod_DF['MONTH']=prod_DF['MONTH'].apply(lambda x: calendar.month_name[round(x)])

#--material_DF--
material_DF.drop_duplicates(subset=['Material2'], ignore_index=True,inplace=True)

#--csmaterial_DF--
csmaterial_DF = csmaterial_DF[['Customer','Material']]
csmaterial_DF.drop_duplicates(subset=['Material'],ignore_index=True,inplace=True)

#--customer_DF--
customer_DF = customer_DF[['Sold2P','Sold2PName','Material']]
customer_DF.drop_duplicates(subset=['Material'], ignore_index=True, inplace=True)
customer_DF.drop_duplicates(subset=['Sold2P'],ignore_index=True, inplace=True)

#--data_DF--
data_DF = prod_DF.merge(material_DF, how='left', left_on='MATERIAL', right_on='Material2')
data_DF = data_DF.merge(csmaterial_DF, how='left', on='Material')
data_DF = data_DF.merge(customer_DF, how='left', left_on='Customer', right_on='Sold2P')
fill_na = {'Customer':0,'Sold2P':0,'Sold2PName':'Unknown','Material':0}
data_DF.fillna(value=fill_na,inplace=True)

## Analysis

In [131]:
prod_by_month = data_DF[['WRKCENTER','MONTH','Quantity']]\
                .groupby(by=['MONTH','WRKCENTER'], as_index=False)\
                ['Quantity'].sum()

prod_by_customer = data_DF[['WRKCENTER','Sold2PName','Quantity']]\
                .groupby(by=['WRKCENTER','Sold2PName'], as_index=False)\
                ['Quantity'].sum()

prod_gap_month = prod_by_month
prod_gap_month['Target'] = 25_000_000
prod_gap_month['Gap'] = prod_gap_month['Target'] - prod_gap_month['Quantity']


#### Saving the data to Excel file 

In [140]:
if os.path.isfile(path_+'Prodana.xlsx'):
    with pd.ExcelWriter(path_+'Prodana.xlsx', mode='a') as writer:
        prod_by_month.to_excel(writer, sheet_name='By_month',index=False)
        prod_by_customer.to_excel(writer, sheet_name='By_customer',index=False)
        prod_gap_month.to_excel(writer, sheet_name='ProdLoss',index=False)
else:
    with pd.ExcelWriter(path_+'Prodana.xlsx') as writer:
        prod_by_month.to_excel(writer, sheet_name='By_month',index=False)
        prod_by_customer.to_excel(writer, sheet_name='By_customer',index=False)
        prod_gap_month.to_excel(writer, sheet_name='ProdLoss',index=False)    